In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [5]:
dbpedia = tf.contrib.learn.datasets.load_dataset(
      'dbpedia', test_with_fake_data=False, size='small')

Successfully downloaded dbpedia_csv.tar.gz 68431223 bytes.
Instructions for updating:
Use tf.data instead.
Instructions for updating:
Use tf.data instead.


In [12]:
x_train = pd.DataFrame(dbpedia.train.data)[1]
y_train = pd.Series(dbpedia.train.target)
x_test = pd.DataFrame(dbpedia.test.data)[1]
y_test = pd.Series(dbpedia.test.target)

In [15]:
print(x_train[0])
print(len(x_train[0]))

 Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.
248


In [14]:
y_train[0]

1

In [17]:
x_train[0]

' Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.'

In [18]:
# Process vocabulary
MAX_DOCUMENT_LENGTH = 100
char_processor = tf.contrib.learn.preprocessing.ByteProcessor(
  MAX_DOCUMENT_LENGTH)
x_train = np.array(list(char_processor.fit_transform(x_train)))
x_test = np.array(list(char_processor.transform(x_test)))

In [19]:
x_train[0]

array([ 32,  65,  98,  98, 111, 116, 116,  32, 111, 102,  32,  70,  97,
       114, 110, 104,  97, 109,  32,  69,  32,  68,  32,  65,  98,  98,
       111, 116, 116,  32,  76, 105, 109, 105, 116, 101, 100,  32, 119,
        97, 115,  32,  97,  32,  66, 114, 105, 116, 105, 115, 104,  32,
        99, 111,  97,  99, 104,  98, 117, 105, 108, 100, 105, 110, 103,
        32,  98, 117, 115, 105, 110, 101, 115, 115,  32,  98,  97, 115,
       101, 100,  32, 105, 110,  32,  70,  97, 114, 110, 104,  97, 109,
        32,  83, 117, 114, 114, 101, 121,  32, 116], dtype=uint8)